# Dùng sentence bert cho bài toán searching dùng embedding (Question Answering).

## Các bước thực hiện:

1. Load data từ database, dùng underthesea để tách câu.
2. Dùng sentence bert (bi-encoder) để tạo embedding cho toàn bộ câu bóc tách. 
3. Compare embedding của câu hỏi với embedding của các câu bóc tách, lấy ra top k câu có embedding gần nhất.
4. Dùng cross encoder để lấy ra 3 câu match nhất với câu hỏi.
5. Load ra 3 câu sau mỗi câu match đúng nhất.

- Dùng code từ file:

https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb

- Document để hiểu hơn về bài toán:
https://aclanthology.org/2022.emnlp-industry.16.pdf


![Bi encoder and cross encoder](images/bi-encoder.png)

In [3]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
import pickle
import pandas as pd

from importlib import reload



/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_38623/3062833682.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import json

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = """SELECT vi.hash_id, vi.contents as question_content, JSON_UNQUOTE(JSON_EXTRACT(tags, '$[*].slug')) AS slug FROM viblo_interview vi"""
df_ques = pd.read_sql(query, connection)

query_ans = """SELECT va.hash_id, va.contents, va.question_id FROM viblo_answer va"""
df_answer = pd.read_sql(query_ans, connection)

# Close the database connection
connection.close()


def filter_slug(x):
    if x is None:
        return ''
    t = str(x).replace("b'", "").replace("'", "")
    t1 = json.loads(t)

    return t1
df_ques['slug_filter'] = df_ques['slug'].apply(filter_slug)
print(df_ques['slug_filter'][0])

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_38623/866573376.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ques = pd.read_sql(query, connection)
/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_38623/866573376.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_answer = pd.read_sql(query_ans, connection)


['javascript']


In [4]:
# This function will search all wikipedia articles for passages that
# answer the query

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

In [5]:
#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [13]:
from underthesea import sent_tokenize

def us_split_sent(text):
    sents = sent_tokenize(text)
    try:
        return sents
    except TypeError:
        return []


def clear_text_vi(posts = []):
    def split_sentences(text):
        try:
            sents = us_split_sent(text)
        except Exception as e:
            print('Error:', e, text)
            sents = []
        sents_1 = [sent for sent in sents if sent.strip() != ""]
        return sents_1

    t = map(split_sentences, posts)
    return list(t)

def flatten_list(l):
    return [item for sublist in l for item in sublist]

In [14]:
data_list = df_ques['question_content'].values.tolist()
sent = clear_text_vi(data_list)

In [15]:
print(len(sent), sent[:2])

3617 [['Thế nào là abstract syntax tree?', 'So với cú pháp Balan thì notation nào sẽ có hiệu năng tốt hơn'], ['Nếu các kỹ thuật thường được sử dụng để trích xuất Credentials trên Windows?']]


In [17]:
sent_flat = [item for sent1 in sent for item in sent1]

print(sent_flat[:2])

['Thế nào là abstract syntax tree?', 'So với cú pháp Balan thì notation nào sẽ có hiệu năng tốt hơn']


In [18]:
corpus_embeddings = bi_encoder.encode(sent_flat, convert_to_tensor=True, show_progress_bar=True)
print(len(corpus_embeddings))

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

4059


In [19]:
#Store sentences & embeddings on disc
with open('./data/sbas-embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': sent_flat, 'embeddings': corpus_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Load sentences & embeddings from disc
with open('./data/sbas-embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    sent_flat = stored_data['sentences']
    corpus_embeddings = stored_data['embeddings']

In [20]:
def get_sent_seq_from_hit(hit, num_after = 3):
    sent_seq = []
    for i in range(num_after):
        # check index exist in sent_flat before append
        if (hit['corpus_id'] + i) < len(sent_flat):
            sent_seq.append(sent_flat[hit['corpus_id'] + i])
    return ' '.join(sent_seq)

def search(query):
    print("Input question:", query)
    ##### Sematic Search #####
    query = clear_text_vi([query])
    # print("Input question clean sentences:", query)
    query = flatten_list(query)
    # print("Input question flattern:", query)

    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    # question_embedding = question_embedding
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query[0], sent_flat[hit['corpus_id']]] for hit in hits]
    
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]


    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:3]:
        print("Score:", hit['cross-score'])
        print("\t " + get_sent_seq_from_hit(hit).replace("\n", " "))

In [98]:
search(query = "Tình hình chứng khoán thế giới trong tháng 9 ?")

Input question: Tình hình chứng khoán thế giới trong tháng 9 ?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
Score: 6.6487575
	 Tính từ khi thị trường chứng khoán Việt Nam được thành lập từ tháng 7-2000, trong tháng 5, chỉ số VN- Index có 12 lần giảm, 9 lần tăng. Trong đó, các năm có tháng 5 giảm mạnh nhất là năm 2008 (giảm 21,7%), 2006 (giảm 12,6%), 2011 (giảm 10,5%), 2012 (giảm 9,2%). Ngược lại, các năm có tháng 5 tăng mạnh nhất là năm 2009 (tăng 26,7%), 2001 (tăng 25,6%), 2007 và 2020 (cùng tăng 15,2%) và 2013 (tăng 9,3%).
Score: 6.6434627
	 Chứng khoán sẽ nối dài đà tăng trong tháng 9?Các chuyên gia nhận định, thị trường chứng khoán (TTCK) vẫn duy trì được đà tăng trong những tháng tới nhờ yếu tố lãi suất cũng như kỳ vọng về kết quả kinh doanh của các doanh nghiệp sẽ tốt hơn trong thời gian tới. Duy trì đà tăng Ông Nguyễn Thế Minh - Giám đốc phân tích khối khách hàng cá nhân CTCK Yuanta Việt Nam nhận định TTCK vẫn có đủ catalyst (yếu tố xúc tác) để thúc đẩy thị trư

In [99]:
search(query = "Giá kim loại, vàng miếng trong tháng 9, năm 2023 ?")

Input question: Giá kim loại, vàng miếng trong tháng 9, năm 2023 ?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
Score: 7.6517096
         Giá vàng miếng SJC tăng 150.000 đồng mỗi lượng vào sáng 30.8. Công ty vàng bạc đá quý Sài Gòn - SJC mua vào lên 67,7 triệu đồng, bán ra 68,3 triệu đồng; Eximbank mua vào với giá 67,8 triệu đồng, bán ra 68,2 triệu đồng… So với đầu tháng 8, kim loại quý đã tăng 1,1 triệu đồng mỗi lượng.
Score: 6.6588287
         Giá vàng miếng SJC sáng ngày 13.6 giảm 300.000 đồng mỗi lượng, Eximbank mua vào với giá 68,2 triệu đồng/lượng và bán ra 69,1 triệu đồng/lượng; Công ty vàng bạc đá quý Sài Gòn – SJC mua vào với giá 68,45 triệu đồng/lượng và bán ra 69,35 triệu đồng/lượng… Tốc độ giảm giá của vàng trong nước nhanh hơn quốc tế khiến SJC cao hơn thế giới còn 17,15 triệu đồng/lượng.
Score: 5.7673564
	 So với giá kim loại quý thế giới, vàng miếng SJC cao hơn 11,7 triệu đồng/lượng, còn nữ trang và nhẫn cao hơn 2,2 - 3,3 triệu đồng/lượng. Giá vàng thế 

In [101]:
search(query = "Kinh tế Trung Quốc trong quí 3 năm 2023 ?")

Input question: Kinh tế Trung Quốc trong quí 3 năm 2023 ?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
Score: 7.5959907
                     // kiem tra xem co la anh khong? == 'div') { năm 2023. Thị trường chứng khoán Mỹ giảm điểm, chỉ số Dow Jones rớt 391,76 điểm (tương đương 1,14%) xuống 33.910,85 điểm; chỉ số S&P 500 mất 0,2% còn 3.990,97 điểm, còn chỉ số Nasdaq Composite giảm 0,14% xuống 11.095,11 điểm. //Chèn ads giữa bài
Score: 7.553033
	 Tại Hội nghị Công tác kinh tế Trung ương thường niên hôm 16/9, các nhà lãnh Trung Quốc đã vạch ra đường hướng cho phát triển kinh tế trong năm 2023. Theo đó, năm tới, nước này sẽ tập trung vào bình ổn nền kinh tế và tiến tới điều chỉnh chính sách để đảm bảo đạt được các mục tiêu chính của nền kinh tế. “Có thể mất ít nhất một quý nữa mọi thứ tại Trung Quốc mới bắt đầu khởi sắc”, ông Dan Wang, nhà kinh tế trưởng tại ngân hàng Hang Seng Bank China, nhận định.
Score: 7.316267
	 Citi trước đó dự báo kinh tế Trung Quốc tăng trưởng 5